In [1]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("data/RAMAYANA.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)


In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os

file = open('data/key.txt', 'r')
# Read the API key from the file
api_key = file.read().strip()
# Close the file
file.close()
# 🧠 Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = api_key

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                        model_kwargs={"device": "cpu"})
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever()


In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
def hitl_query(question):
    result = qa_chain(question)
    answer = result['result']
    sources = result['source_documents']

    print(f"\n🧠 AI Answer: {answer}\n")
    print("📄 Source Chunks Used:")
    for i, doc in enumerate(sources):
        print(f"  {i+1}. Page {doc.metadata.get('page', 'unknown')} — {doc.page_content[:150]}...\n")

    feedback = input("✅ Is the answer correct? (yes/no): ").strip().lower()

    if feedback == 'no':
        correct_answer = input("❓ What is the correct answer?: ")
        with open("hitl_feedback_log.txt", "a", encoding="utf-8") as f:
            f.write(f"Question: {question}\n")
            f.write(f"AI Answer: {answer}\n")
            f.write(f"Correct Answer (by Human): {correct_answer}\n")
            f.write(f"{'-'*40}\n")
        print("💾 Feedback saved for review.")
    else:
        print("✅ Thank you! Marked as correct.")


In [9]:
hitl_query("Who was the father of Lord Rama?")



🧠 AI Answer: The father of Lord Rama was King Dasharatha.

📄 Source Chunks Used:
  1. Page 19 — Rama embraced his brothers and was concerned to see agony on 
their faces. So he asked Bharatha “How is father? Have you taken 
over the affairs of ki...

  2. Page 11 — Rama!” 
“Mother, why is father looking so pale and grief-stricken? Have I 
caused him any anguish?” asked Rama politely. 
Kaikeyi told him about her t...

  3. Page 17 — Hadn’t you loved him more than you loved me? Alas! You have killed 
my father, just to make me the king. And do you expect me to wear 
the crown that ...

  4. Page 4 — 4 
 
because he himself is very powerful. He wants to take Rama with 
him so that your son can learn to fight the Rakshasas even at this 
young age. S...

✅ Thank you! Marked as correct.
